# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Признаки:

DateCrawled — дата скачивания анкеты из базы

VehicleType — тип автомобильного кузова

RegistrationYear — год регистрации автомобиля

Gearbox — тип коробки передач

Power — мощность (л. с.)

Model — модель автомобиля

Kilometer — пробег (км)

RegistrationMonth — месяц регистрации автомобиля

FuelType — тип топлива

Brand — марка автомобиля

Repaired — была машина в ремонте или нет

DateCreated — дата создания анкеты

NumberOfPictures — количество фотографий автомобиля

PostalCode — почтовый индекс владельца анкеты (пользователя)

LastSeen — дата последней активности пользователя

Целевой признак:

Price — цена (евро)

## Подготовка данных

In [ ]:
#!pip install pandas
#!pip install matplotlib
#!pip install seaborn
#!pip install statsmodels
#!pip install sklearn
#!pip install time
!pip install scikit-learn==1.1.3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy
import math
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.compose import make_column_transformer
from catboost import Pool, CatBoostRegressor, cv
from lightgbm import LGBMRegressor


from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split,
    cross_val_score
)

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler,
    PolynomialFeatures
)

RANDOM_STATE = 42

In [ ]:
data = pd.read_csv('/datasets/autos.csv')

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
# Посчитаем сколько пропусков в столбцах и оценим их кол-во
print(data.isna().sum())
data.isna().mean()

Пропуски есть в столбцах VehicleType, Gearbox, Model, FuelType и Repaired. 

In [ ]:
# Создадим функцию, которая выведет для каждого столбца уникальные значения
def unique_data(data_frame):
    for column in data_frame.columns:
        print('Уникальные значения столбца:', column)
        print(data_frame[column].unique())

In [ ]:
# Вызовем функции для наших данных
unique_data(data)

В столбце RegistrationYear присутствуют некорректные значения наподобие 5300-го года.

В столбце Power есть некорректные значения мощности по типу 10к л.с. или слишком маленькие значения.

В столбце RegistrationMonth есть нулевой месяц регистрации, то есть отсутствует месяц регистрации.

Признаки/столбцы, которые влияют на стоимость машины:

VehicleType. Тип автомобильного кузова. Чем лучше кузов, тем выше цена. 

Gearbox. Тип коробки передач. Наличие автоматической коробки передач повышает стоимость - чем более сложная конструкция, тем выше цена. 

Power. Мощность в лошадиных силах влияет на стоимость.

Model. Модель машины. Влияет на востребованность и на стоимость. 

Kilometer. Чем больше пробег, тем меньше цена. Связь обратно пропорциональная.

FuelType. Тип топлива. Из этого можно сделать вывод какой двигатель, что тоже влияет на стоимость.

Brand. Марка автомобиля. Говорит о качестве, надежности, а также статусе, что влияет на стоимость.

Repaired. Машины, которые побывали в ремонте, то есть они подвергались изменениям. Надежность ниже, неизвестно, что заменяли.

RegistrationYear. Год регистрации. Чем больше возраст авто, тем меньше цена.

Столбцы: DateCrawled — дата скачивания анкеты из базы, DateCreated — дата создания анкеты, NumberOfPictures — количество фотографий автомобиля, PostalCode — почтовый индекс владельца анкеты (пользователя), LastSeen — дата последней активности пользователя

Предобработка данных

In [ ]:
# Ограничимся столбцами, которые влияют на стоимость
new_data = data[['VehicleType', 
                         'Gearbox', 
                         'Power',
                         'Kilometer',
                         'FuelType',
                         'Brand',
                         'Repaired',
                         'RegistrationYear',
                         'Model',
                         'Price']]
new_data.info()

Столбцы, требующие обработки:

VehicleType - Восстановим тип автомобильного кузова по типу из столбца Brand. 

Gearbox - Для всех NaN установим ручную коробку передач - 'manual'.

Power - Значения около 1 л.с. или 10к л.с. для машины некорректны. Скорректируем значения на основе реальных. 

FuelType - Восстановим пропущенные значения по популярному типу из столбца Brand.

Repaired - Все NaN заменим на 'no'.

RegistrationYear. Скоректируем на основе реальных значений в машинах.

Model. Для всех NaN установим значение 'unknown'.

In [ ]:
# Изучим значения столбца Price
new_data['Price'].value_counts()

Видим, что пропущено 10772 значений. Удалим их и представим распределение цен на гистограмме.

In [ ]:
new_data = new_data.query('Price > 0')
new_data['Price'].hist(bins=100, figsize=(12,9))
new_data['Price'].describe()

500 евро - это 50000 рублей, вполне можно купить поддержанный авто за эти деньги. 

In [ ]:
new_data = new_data.query('500 <= Price')
new_data.info()

In [ ]:
# Представим значения RegistrationYear на гистограмме.
new_data['RegistrationYear'].hist(bins=100, figsize=(12,9))
new_data['RegistrationYear'].describe()

Гистограмма не слишком наглядна, но присутствуют выбросы в годах регистрации. Отсечем значения, исходя из максимальной даты скачивания анкеты автомобиля.

In [ ]:
date = pd.to_datetime(data['DateCrawled'])
date.max()

Таким образом, отсекаем все значения после 2016, а также, машины появились не раньше 1900 года. 

In [ ]:
new_data = new_data.query('1899 < RegistrationYear < 2017')
new_data['RegistrationYear'].hist(bins=100, figsize=(12,9))
new_data['RegistrationYear'].describe()

Исходя из гистограммы видно, что можно ограничиться левой границей в 1960 год. 

In [ ]:
new_data = new_data.query('1959 < RegistrationYear < 2017')
new_data['RegistrationYear'].hist(bins=100, figsize=(12,9))
new_data['RegistrationYear'].describe()

In [ ]:
# Заменим значения NaN в Gearbox, Repaired и Model.
new_data.fillna({'Gearbox':'manual', 
                           'Repaired':'no',
                           'Model':'unknown'},
                          inplace=True)
new_data.isna().mean()

In [ ]:
# В столбце VehicleType оценим количество пропущенных значений относительно всех значений в этом столбце
display(new_data['VehicleType'].value_counts())
display(new_data['VehicleType'].isna().sum())

In [ ]:
# Построим сводную таблицу, чтобы увидеть у каких брендов какие модели, кузов и сколько их.
vehicle_type_pivot = new_data.pivot_table(index=['VehicleType'], 
                                                    columns=['Brand'],
                                                    values=['Price'],
                                                    aggfunc=['count'])
display(vehicle_type_pivot)

In [ ]:
vehicle_type_pivot = vehicle_type_pivot['count']['Price']
vehicle_type_pivot

In [ ]:
new_data.info()

In [ ]:
# Заполним пустые ячейки нулями. И напишем функцию, которая будет пропущенные значения заменять на наиболее популярный тип 
# кузова для конкретной марки авто 
new_data.fillna({'VehicleType':0},inplace=True)

def fillna_by_brand(row, row_number, vehicle_type_pivot):
    brand = row[5]
    fill_value = row[row_number]
    if row[row_number] == 0:
        fill_value = vehicle_type_pivot[brand].idxmax()
    return fill_value

new_data['VehicleType'] = new_data.apply(
                                fillna_by_brand, 
                                args=[0, vehicle_type_pivot],
                                axis=1)
display(new_data['VehicleType'].isna().sum())
new_data['VehicleType'].value_counts()

In [ ]:
# Аналогично заполним пропуски для столбца FuelType
new_data['FuelType'].value_counts()
fuel_type_pivot = new_data.pivot_table(index=['FuelType'], 
                                                          columns=['Brand'],
                                                          values=['Price'],
                                                          aggfunc=['count'])
fuel_type_pivot = fuel_type_pivot['count']['Price']
display(fuel_type_pivot)

new_data.fillna({'FuelType':0},inplace=True)

new_data['FuelType'] = new_data.apply(
                                fillna_by_brand, 
                                args=[4, vehicle_type_pivot],
                                axis=1)
display(new_data['FuelType'].isna().sum())
new_data['FuelType'].value_counts()

In [ ]:
new_data.isna().mean()

In [ ]:
# Теперь обработаем некорректные значения в столбце Power. Рассмотрим распределение значений на гистограмме
new_data['Power'].hist(bins=100, figsize=(12,9))
new_data['Power'].describe()

Очень сильный разброс. Уберем выбросы больше 1000 л.с.

In [ ]:
new_data = new_data.query('Power < 1001')
new_data['Power'].hist(bins=100, figsize=(12,9))
new_data['Power'].describe()

Исходя из графика, мы видим, что у многих машин мощность равна 0. Удалим эти строки. А в качестве верхней границы выберем 400 л.с. 

In [ ]:
new_data = new_data.query('0 < Power < 401')
new_data['Power'].hist(bins=100, figsize=(12,9))
new_data['Power'].describe()

Исходя из малого кол-ва или отсутствия строк, ограничим еще - слева отсечем по 25 и справа по 350 л.с.

In [ ]:
good_new_data = new_data.query('24 < Power < 401')
good_new_data['Power'].hist(bins=100, figsize=(12,9))
good_new_data['Power'].describe()

In [ ]:
good_new_data.info()

Заменем тип значений в столбцах Gearbox и Repaired на булев тип.

IsGearbox. 1(manual) - механическая коробка передач. 0(auto) - автоматическая.
IsRepaired. 1(yes) - машина с ремонтом, 0(no) - без ремонта.

In [ ]:
good_new_data['IsManualGearbox'] = 0
good_new_data.loc[good_new_data['Gearbox'] == 'manual', 'IsManualGearbox'] = 1
display(good_new_data['IsManualGearbox'].value_counts())
good_new_data['Gearbox'].value_counts()

In [ ]:
good_new_data['IsRepaired'] = 1
good_new_data.loc[good_new_data['Repaired'] == 'yes', 'IsRepaired'] = 0
display(good_new_data['IsRepaired'].value_counts())
good_new_data['Repaired'].value_counts()

Удалим столбцы, которые не влияют на стоимость и поменяем тип для того, чтобы было быстрее работать с данными

In [ ]:
good_new_data.drop(['Repaired','Gearbox'], inplace=True, axis=1)
good_new_data['IsRepaired'] = pd.to_numeric(good_new_data['IsRepaired'], downcast='integer')
good_new_data['IsManualGearbox'] = pd.to_numeric(good_new_data['IsManualGearbox'], downcast='integer')
good_new_data['RegistrationYear'] = pd.to_numeric(good_new_data['RegistrationYear'], downcast='integer')
good_new_data['Power'] = pd.to_numeric(good_new_data['Power'], downcast='integer')
good_new_data['Kilometer'] = pd.to_numeric(good_new_data['Kilometer'], downcast='integer')
good_new_data['Price'] = pd.to_numeric(good_new_data['Price'], downcast='integer')
good_new_data.info()

Вывод:

    Изучили данные, выделили столбцы, влияющие на стоимость, которые будем использовать для обучения модели.
    
    Обработали пропущенные и некорректные значения.
    
    Заменили два категориальных признака на численные.

## Обучение моделей

Разделим датасет на тренировочную и тестовую выборки.

In [ ]:
features = good_new_data.drop(['Price'], axis=1)
target = good_new_data['Price']

X_train, X_test, y_train, y_test = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE
)

Перейдем к кодированию и масштабированию данных. 

Для модели Ridge - будем использовать OHE кодирование категориальных признаков  и StandardScaler для численных признаков.

Для модели RandomForestRegressor, буде использовать порядковое кодирование признака model, имеющего большое количество уникальных значений, и OHE кодирование остальных категориальных признаков.

А также, рассмотрим модель LightGBM. Подберем гиперпараметры для нее с помощью GridSearchCV

In [ ]:
#категориальные признаки для OHE Ridge
ohe_features_ridge = X_train.select_dtypes(include='object').columns.to_list()
print(ohe_features_ridge)

#категориальные признаки для OHE RandomForestRegressor
ohe_features_rf = ohe_features_ridge.copy()
ohe_features_rf.remove('Model')

#категориальные признаки для OHE LGBM
ohe_features_LGBM = ohe_features_ridge.copy()
ohe_features_LGBM.remove('Model')

In [ ]:
#численные признаки
# Исключаем repaired, так как это категориальный бинарный признак.
num_features = X_train.select_dtypes(exclude='object').columns.to_list()
num_features.remove('IsRepaired')
num_features

### Ridge

In [ ]:
X_train_ridge = X_train.copy()
X_test_ridge = X_test.copy()

In [ ]:
# признак repaired уже бинарный, его не будем кодировать/масштабировать
# добавляем remainder='passthrough, чтобы он не пропал
col_transformer_ridge = make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'),
        ohe_features_ridge
    ),
    (
        StandardScaler(),
        num_features
    ),
    remainder='passthrough',
    #verbose_feature_names_out=False
)

# всё готово в пару строк кода
X_train_ridge = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_ridge.fit_transform(X_train_ridge),
    columns=col_transformer_ridge.get_feature_names_out()
)

# смотрим на результат
X_train_ridge.head()

In [ ]:
# трансформируем тестовую выборку
X_test_ridge = pd.DataFrame.sparse.from_spmatrix(
    col_transformer_ridge.transform(X_test_ridge),
    columns=col_transformer_ridge.get_feature_names_out()
)

# смотрим на результат
X_test_ridge.head()

Подберем гиперпараметры с помощью GridSearchCV, который имеет встроенную кросс-валидацию для модели Ridge

In [ ]:
X_train_ridge = X_train.copy()

In [ ]:
# random_state не перебирается, задаём его прямо в модели
model_ridge = Ridge(random_state=RANDOM_STATE)

pipeline_ridge = make_pipeline(col_transformer_ridge, model_ridge)

# словарь с гиперпараметрами и значениями, которые хотим перебрать
param_grid_ridge = {
    'ridge__alpha': np.arange(0, 0.21, 0.01),
}

gs_ridge_pl = GridSearchCV(
    pipeline_ridge,
    param_grid=param_grid_ridge,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)


In [ ]:
%%time
gs_ridge_pl.fit(X_train_ridge, y_train)

In [ ]:
gs_ridge_best_score = gs_ridge_pl.best_score_ * -1
gs_ridge_best_params = gs_ridge_pl.best_params_

# лучшее значение RMSE на кросс-валидации
print(f'best_score: {gs_ridge_best_score}')
# лучшие гиперпараметры
print(f'best_params: {gs_ridge_best_params}')

### RandomForestRegressor

In [ ]:
X_train_rf = X_train.copy()
X_test_rf = X_test.copy()

In [ ]:
col_transformer_rf= make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'),
        ohe_features_rf
    ),
    (
        OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),
        ['Model']
    ),
    (
        StandardScaler(),
        num_features
    ),
    remainder='passthrough'
)

model_rf = RandomForestRegressor(random_state=RANDOM_STATE)

pipline_rf = make_pipeline(col_transformer_rf, model_rf)

param_grid_rf = {
    'randomforestregressor__n_estimators': range(50, 251, 50),
    'randomforestregressor__max_depth': range(2, 15),
    'randomforestregressor__min_samples_split': (2, 3, 4),
    'randomforestregressor__min_samples_leaf': (1, 2, 3, 4)
}

# получается достаточно много комбинаций гиперпараметров при переборе
# будем использовать RandomizedSearchCV, он работает на много быстрее
gs_rf = RandomizedSearchCV(
    pipline_rf,
    param_distributions=param_grid_rf,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    random_state=RANDOM_STATE
)

In [ ]:
%%time
gs_rf.fit(X_train_rf, y_train)

In [ ]:
gs_rf_best_score = gs_rf.best_score_ * -1
gs_rf_best_params = gs_rf.best_params_
print(f'best_score: {gs_rf_best_score}')
print(f'best_params: {gs_rf_best_params}')

In [ ]:
result = pd.DataFrame(
    [gs_ridge_best_score, gs_rf_best_score],
    index=['Ridge', 'RandomForestRegressor'],
    columns=['RMSE']
)
result

LGBMRegressor

Обучение модели

In [ ]:
X_train_LGBM = X_train.copy()
X_test_LGBM = X_test.copy()

In [ ]:
col_transformer_LGBM = make_column_transformer(
    (
        OneHotEncoder(drop='first', handle_unknown='ignore'),
        ohe_features_LGBM
    ),
    (
        OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1),
        ['Model']
    ),
    (
        StandardScaler(),
        num_features
    ),
    remainder='passthrough'
)

model_LGBM = LGBMRegressor() 

pipline_LGBM = make_pipeline(col_transformer_LGBM, model_LGBM)

param_grid_LGBM = {
    'lgbmregressor__num_leaves': (31,100,200),
    'lgbmregressor__learning_rate': (0.1, 0.3, 0.5)
}


gs_LGBM = RandomizedSearchCV(
    pipline_LGBM,
    param_distributions=param_grid_LGBM,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    random_state=RANDOM_STATE
)

In [ ]:
%%time
gs_LGBM.fit(X_train_LGBM, y_train)

In [ ]:
gs_LGBM_best_score = gs_LGBM.best_score_ * -1
gs_LGBM_best_params = gs_LGBM.best_params_
print(f'best_score: {gs_LGBM_best_score}')
print(f'best_params: {gs_LGBM_best_params}')

In [ ]:
result = pd.DataFrame(
    [gs_ridge_best_score, gs_rf_best_score, gs_LGBM_best_score],
    index=['Ridge', 'RandomForestRegressor', 'LGBM'],
    columns=['RMSE']
)
result

Вывод:
Были изучены 3 регрессионные модели. 
Найдены лучшие гиперпараметры и время выполнения

## Анализ моделей

Проверим качество LGBMRegressor на тестовой выборке:

In [ ]:
%%time
# Сделаем предсказание на тестовой выборке
y_pred = gs_LGBM.predict(X_test_LGBM)

In [ ]:
# Вычислим RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', rmse)

Итоговый вывод:

Загружены данные. Выполнена предобработка пропущенных значений и некорректных значений.

Сравнили модели с различными наборами гиперпараметров.

Выбрана лучшая модель по результатам метрики RMSE и времени обучения.

Если нужна точность и не так важно время, LGBMRegressor показала лучшие результаты RMSE - 1525.7480449456173